# 面向目标架构案例实践

## 1. 场景数据准备(收邮件，写邮件)

步骤1：安装MySQL数据库

sudo apt-get update

sudo apt-get install mysql-server

sudo service mysql start

步骤2：创建数据库用户

CREATE USER 'gpt'@'localhost' IDENTIFIED BY 'gpt';

步骤3：给数据库用户赋权限

GRANT ALL PRIVILEGES ON *.* TO 'gpt'@'localhost';

FLUSH PRIVILEGES;


In [ ]:
create database my_mail;
use my_mail;
CREATE TABLE mailbox (
    userid VARCHAR(255),
    from_userid VARCHAR(255),
    date VARCHAR(255),
    subject VARCHAR(255),
    snippet VARCHAR(255), 
    status VARCHAR(255),
    label VARCHAR(255)
);

In [ ]:
INSERT INTO mailbox (userid, from_userid, date, subject, snippet, status, label)
VALUES ('me', '西门吹雪', '2024-06-06', '速回公司', '赶快回来公司', '未读', '开会');

INSERT INTO mailbox (userid, from_userid, date, subject, snippet, status, label)
VALUES ('me', '西门吹雪', '2024-06-07', '开会', '明天10点到海淀区叶青大厦5楼502开会', '已读', '开会');

INSERT INTO mailbox (userid, from_userid, date, subject, snippet, status, label)
VALUES ('西门吹雪', 'me', '2024-06-07', '开会', '收到，准时参加', '已读', '开会');

INSERT INTO mailbox (userid, from_userid, date, subject, snippet, status, label)
VALUES ('me', '陆小凤', '2024-06-03', '2024年Q2规划', 'Q2规划：聚焦核心业务，优化流程，提升效率。加强团队协作，培养人才。探索新市场，拓宽业务领域。确保目标达成，实现增长。详细见附件', '未读', '规划');


In [ ]:
UPDATE mailbox
SET snippet = '李先生，您好，我是陆小凤，这是我给您发的邮件，里面是Q2规划：聚焦核心业务，优化流程，提升效率。加强团队协作，培养人才。探索新市场，拓宽业务领域。确保目标达成，实现增长。详细见邮箱附件'
WHERE userid = 'me' and from_userid='陆小凤'

## 2. 体验大模型的外部函数能力

In [ ]:
## GLM-4大模型

In [36]:
import pymysql
import numpy as np
import pandas as pd

def retrieve_emails(n,user_id='me'):
    """
    获取指定数量的最近邮件。
    参数:
    n: 要检索的邮件的数量,数据格式为int类型。
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{user_id}' ORDER BY date DESC LIMIT {n}";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")     

In [7]:
msg = retrieve_emails(2)

In [8]:
msg

'[{"userid":"me","from_userid":"\\u897f\\u95e8\\u5439\\u96ea","date":"2024-06-07","subject":"\\u5f00\\u4f1a","snippet":"\\u660e\\u592910\\u70b9\\u5230\\u6d77\\u6dc0\\u533a\\u53f6\\u9752\\u5927\\u53a65\\u697c502\\u5f00\\u4f1a","status":"\\u5df2\\u8bfb","label":"\\u5f00\\u4f1a"},{"userid":"me","from_userid":"\\u897f\\u95e8\\u5439\\u96ea","date":"2024-06-06","subject":"\\u901f\\u56de\\u516c\\u53f8","snippet":"\\u8d76\\u5feb\\u56de\\u6765\\u516c\\u53f8","status":"\\u672a\\u8bfb","label":"\\u5f00\\u4f1a"}]'

In [50]:
import os
import openai
from openai import OpenAI
import shutil

import json
import io
import inspect
import requests
import re
import random
import string

## 初始化客户端
api_key = os.getenv("ZHIPU_API_KEY")
#api_key="4abxxUM"
## pip install zhipuai

from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key)

In [10]:
messages=[
            {"role": "system", "content": "这是我的邮箱最近2封邮件的内容：%s" % msg},
            {"role": "system", "content": "邮件内容是由从my_mail库的mailbox表中获取的"},
            {"role": "user", "content": "请问我的最近2封邮件是谁发送的，都有什么内容？"}
          ]
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages)

response_message = response.choices[0].message

In [13]:
print(response_message.content)

根据您提供的邮箱内容信息，以下是两封邮件的概要：

1. 第一封邮件：
   - 发件人：西門吹雪（根据中文字符转义，实际可能是“西门吹雪”）
   - 日期：2024年6月7日
   - 主题：开会
   - 邮件摘要：明天上午10点到海棠区杉青大厦5楼502室开会。
   - 状态：已读
   - 标签：开会

2. 第二封邮件：
   - 发件人：西門吹雪（同样，可能是“西门吹雪”）
   - 日期：2024年6月6日
   - 主题：速回公司
   - 邮件摘要：赶快回到公司。
   - 状态：未读
   - 标签：开会

这些邮件内容是从您的邮箱数据库（my_mail库的mailbox表）中获取的。

至于参考信息部分，第一项是关于一个人收到疑似诈骗邮件的咨询及其回复，这和您的邮件内容没有直接关系。第二项是一个假定场景，要求用英文写一封关于关心朋友Tom和他母亲病情的邮件，这也与您邮件内容无关。不过，如果您需要关于如何回复类似的邮件或者如何用英文撰写邮件的建议，可以提供帮助。


In [44]:
def auto_functions(functions_list):
    """
    Chat模型的functions参数编写函数
    :param functions_list: 包含一个或者多个函数对象的列表；
    :return：满足Chat模型functions参数要求的functions对象
    """
    def functions_generate(functions_list):
        # 创建空列表，用于保存每个函数的描述字典
        functions = []
        # 对每个外部函数进行循环
        for function in functions_list:
            # 读取函数对象的函数说明
            function_description = inspect.getdoc(function)
            # 读取函数的函数名字符串
            function_name = function.__name__
            ## 提示词 one-shot
            system_prompt1="""输出结果必须是一个JSON格式的字典,举个例子：
            [{'type': 'function', 'function': {'name': 'sunwukong_function', 'description': '该函数定义了数据集计算过程，接收一个数据表字符串参数，并返回一个JSON格式的Dataframe类型对象。', 'parameters': {'type': 'object', 'properties': {'data': {'type': 'string', 'description': '带入计算的数据表，用字符串进行表示'}}, 'required': ['data'], 'additionalProperties': False}}}]
            """
            system_prompt2 = '以下是某的函数说明：%s,输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_description
            user_prompt = '根据这个函数的函数说明，请帮我创建一个JSON格式的字典，这个字典有如下5点要求：\
                           1.字典总共有三个键值对；\
                           2.第一个键值对的Key是字符串name，value是该函数的名字：%s，也是字符串；\
                           3.第二个键值对的Key是字符串description，value是该函数的函数的功能说明，也是字符串；\
                           4.第三个键值对的Key是字符串parameters，value是一个JSON Schema对象，用于说明该函数的参数输入规范。\
                           5.输出结果必须是一个JSON格式的字典，只输出这个字典即可，前后不需要任何前后修饰或说明的语句' % function_name

            response = client.chat.completions.create(
                              model="glm-4",
                              messages=[
                                {"role": "system", "content": system_prompt1},
                                {"role": "system", "content": system_prompt2},
                                {"role": "user", "content": user_prompt}
                              ]
                            )
            json_str=response.choices[0].message.content.replace("```json","").replace("```","")
            json_function_description=json.loads(json_str)
            json_str={"type": "function","function":json_function_description}
            functions.append(json_str)
        return functions
    ## 最大可以尝试4次
    max_attempts = 4
    attempts = 0

    while attempts < max_attempts:
        try:
            functions = functions_generate(functions_list)
            break  # 如果代码成功执行，跳出循环
        except Exception as e:
            attempts += 1  # 增加尝试次数
            print("发生错误：", e)
            if attempts == max_attempts:
                print("已达到最大尝试次数，程序终止。")
                raise  # 重新引发最后一个异常
            else:
                print("正在重新运行...")
    return functions

In [45]:
functions_list=[retrieve_emails]

In [46]:
functions = auto_functions(functions_list)
functions

[{'type': 'function',
  'function': {'name': 'retrieve_emails',
   'description': '获取指定数量的最近邮件。',
   'parameters': {'type': 'object',
    'properties': {'n': {'type': 'integer', 'description': '要检索的邮件的数量'},
     'user_id': {'type': 'string',
      'description': '要检索邮件的用户的ID',
      'default': 'me'}},
    'required': ['n'],
    'additionalProperties': False}}}]

In [47]:
def run_conversation(messages, functions_list=None, model="glm-4"):
    """
    能够自动执行外部函数调用的对话模型
    :param messages: 必要参数，字典类型，输入到Chat模型的messages参数对象
    :param functions_list: 可选参数，默认为None，可以设置为包含全部外部函数的列表对象
    :param model: Chat模型，可选参数，默认模型为glm-4
    :return：Chat模型输出结果
    """
    # 如果没有外部函数库，则执行普通的对话任务
    if functions_list == None:
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        )
        response_message = response.choices[0].message
        final_response = response_message.content
        
    # 若存在外部函数库，则需要灵活选取外部函数并进行回答
    else:
        # 创建functions对象
        tools = auto_functions(functions_list)
        
        # 创建外部函数库字典
        available_functions = {func.__name__: func for func in functions_list}
        print(available_functions)

        # 第一次调用大模型
        response = client.chat.completions.create(
                        model=model,
                        messages=messages,
                        tools=tools,
                        tool_choice="auto", )
        response_message = response.choices[0].message
        
       


        tool_calls = response_message.tool_calls

        if tool_calls:

            #messages.append(response.choices[0].message)
            ## GLM-4 Function Call的使用方式
            messages.append(response.choices[0].message.model_dump())
            for tool_call in tool_calls:
                function_name = tool_call.function.name
                function_to_call = available_functions[function_name]
                function_args = json.loads(tool_call.function.arguments)
                ## 真正执行外部函数的就是这儿的代码
                function_response = function_to_call(**function_args)
                messages.append(
                    {
                        "role": "tool",
                        "content": function_response,
                        "tool_call_id": tool_call.id,
                    }
                ) 
            ## 第二次调用模型
            second_response = client.chat.completions.create(
                model=model,
                messages=messages,
                tools=tools
            ) 
            # 获取最终结果
            print(second_response.choices[0].message)
            final_response = second_response.choices[0].message.content
        else:
            final_response = response_message.content
                
    return final_response

In [48]:
messages=[
    {"role": "system", "content": "你是一个邮件助手，帮我查询各种邮件信息"},
    {"role": "user", "content": "帮我查询一下，我最近的3封邮件内容"}
]

In [49]:
result = run_conversation(messages, functions_list=functions_list, model="glm-4")
print(result)

{'retrieve_emails': <function retrieve_emails at 0x7ff852682ee0>}
CompletionMessage(content='您最近的3封邮件内容如下：\n\n1. 日期：2024-06-07\n   发件人：西门吹雪\n   主题：开会\n   内容摘要：明天10点到海棠区玉兰大饭店5楼502开会\n   邮件状态：已读\n   标签：开会\n\n2. 日期：2024-06-06\n   发件人：西门吹雪\n   主题：速回公司\n   内容摘要：赶快回来公司\n   邮件状态：未读\n   标签：开会\n\n3. 日期：2024-06-03\n   发件人：陆小凤\n   主题：2024年Q2规划\n   内容摘要：Q2规划：聚焦核心业务，优化流程，提升效率。加强团队合作，培养人才。探索新市场，拓展业务领域。确保目标达成，实现增长。详细见附件\n   邮件状态：未读\n   标签：规划\n\n请注意查收。如果您需要阅读详细内容或回复，请告诉我。', role='assistant', tool_calls=None)
您最近的3封邮件内容如下：

1. 日期：2024-06-07
   发件人：西门吹雪
   主题：开会
   内容摘要：明天10点到海棠区玉兰大饭店5楼502开会
   邮件状态：已读
   标签：开会

2. 日期：2024-06-06
   发件人：西门吹雪
   主题：速回公司
   内容摘要：赶快回来公司
   邮件状态：未读
   标签：开会

3. 日期：2024-06-03
   发件人：陆小凤
   主题：2024年Q2规划
   内容摘要：Q2规划：聚焦核心业务，优化流程，提升效率。加强团队合作，培养人才。探索新市场，拓展业务领域。确保目标达成，实现增长。详细见附件
   邮件状态：未读
   标签：规划

请注意查收。如果您需要阅读详细内容或回复，请告诉我。


## 3. 借助大模型实现本地代码编写

In [51]:
from zhipuai import ZhipuAI
client = ZhipuAI(api_key=api_key)

In [58]:
messages=[{"role": "system", "content": "你是一个python代码编辑器，你的功能是输出python代码，只输出代码即可，请勿输出任何其他内容"},
            {"role": "user", "content": "请帮我定义一个python函数，输出Hello world字符串，请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档。"}
  ]
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages)

content = response.choices[0].message.content

In [59]:
print(content)

```python
def print_hello_world():
    """
    输出"Hello world"字符串。

    这个函数没有参数，当被调用时会直接打印出"Hello world"字符串。
    这是一个简单的示例函数，常用于编程初学者的第一个程序示例。

    返回值:
        无返回值，直接通过print函数输出到控制台。
    """
    print("Hello world")
```

请注意，根据您的要求，以上代码包含了一个函数定义，以及该函数内部的中文说明文档。当调用`print_hello_world()`函数时，它会输出"Hello world"到控制台。


In [60]:
with open('code/helloworld.md', 'a', encoding='utf-8') as f:
    f.write(content)

In [61]:
def extract_code(s):
    """
    如果输入的字符串s是一个包含Python代码的Markdown格式字符串，提取出代码部分。
    否则，返回原字符串。

    参数:
    s: 输入的字符串。

    返回:
    提取出的代码部分，或原字符串。
    """
    # 判断字符串是否是Markdown格式
    if '```python' in s or 'Python' in s or'PYTHON' in s:
        # 找到代码块的开始和结束位置
        code_start = s.find('def')
        code_end = s.find('```\n', code_start)
        # 提取代码部分
        code = s[code_start:code_end]
    else:
        # 如果字符串不是Markdown格式，返回原字符串
        code = s

    return code

In [62]:
code_python = extract_code(content)
code_python

'def print_hello_world():\n    """\n    输出"Hello world"字符串。\n\n    这个函数没有参数，当被调用时会直接打印出"Hello world"字符串。\n    这是一个简单的示例函数，常用于编程初学者的第一个程序示例。\n\n    返回值:\n        无返回值，直接通过print函数输出到控制台。\n    """\n    print("Hello world")\n'

In [63]:
with open('code/helloworld.py', 'w', encoding='utf-8') as f:
    f.write(code_python)

In [64]:
with open('code/helloworld.py', encoding='utf-8') as f:
    content = f.read()
print(content)

def print_hello_world():
    """
    输出"Hello world"字符串。

    这个函数没有参数，当被调用时会直接打印出"Hello world"字符串。
    这是一个简单的示例函数，常用于编程初学者的第一个程序示例。

    返回值:
        无返回值，直接通过print函数输出到控制台。
    """
    print("Hello world")



In [65]:
exec(content)

In [66]:
print_hello_world()

Hello world


接下来我们定义一个功能更加完整的extract_function_code函数。该函数可以同时在字符串中提取python代码，并提取该段代码的函数名称，同时对该函数进行同名py文件的本地保存，并在当前环境下执行该代码（定义该函数），同时可以选择打印函数的全部信息或者只是打印函数的名称：

In [67]:
def extract_function_code(s_python, detail=0):
    """
    函数提取函数，同时执行函数内容，可以选择打印函数信息
    """
    def extract_code(s_python):
        """
        如果输入的字符串s是一个包含Python代码的Markdown格式字符串，提取出代码部分。
        否则，返回原字符串。

        参数:
        s: 输入的字符串。

        返回:
        提取出的代码部分，或原字符串。
        """
        # 判断字符串是否是Markdown格式
        if '```python' in s_python or 'Python' in s_python or'PYTHON' in s_python:
            # 找到代码块的开始和结束位置
            code_start = s_python.find('def')
            code_end = s_python.find('```\n', code_start)
            # 提取代码部分
            code = s_python[code_start:code_end]
        else:
            # 如果字符串不是Markdown格式，返回原字符串
            code = s_python

        return code
    
    # 提取代码字符串
    code = extract_code(s_python)
    
    # 提取函数名称
    match = re.search(r'def (\w+)', code)
    function_name = match.group(1)
    
    # 将函数写入本地
    with open('%s.py' % function_name, 'w', encoding='utf-8') as f:
        f.write(code)
    
    # 执行该函数
    try:
        exec(code, globals())
    except Exception as e:
        print("An error occurred while executing the code:")
        print(e)
    
    # 打印函数名称
    if detail == 0:
        print("The function name is:%s" % function_name)
    
    if detail == 1:
        with open('%s.py' % function_name, encoding='utf-8') as f:
            content = f.read()
        print(content)

In [68]:
content

'def print_hello_world():\n    """\n    输出"Hello world"字符串。\n\n    这个函数没有参数，当被调用时会直接打印出"Hello world"字符串。\n    这是一个简单的示例函数，常用于编程初学者的第一个程序示例。\n\n    返回值:\n        无返回值，直接通过print函数输出到控制台。\n    """\n    print("Hello world")\n'

In [69]:
extract_function_code(content)

The function name is:print_hello_world


In [70]:
print_hello_world?

Signature: print_hello_world()
Docstring:
输出"Hello world"字符串。

这个函数没有参数，当被调用时会直接打印出"Hello world"字符串。
这是一个简单的示例函数，常用于编程初学者的第一个程序示例。

返回值:
    无返回值，直接通过print函数输出到控制台。
File:      Dynamically generated function. No source code available.
Type:      function

In [71]:
extract_function_code(content, detail=1)

def print_hello_world():
    """
    输出"Hello world"字符串。

    这个函数没有参数，当被调用时会直接打印出"Hello world"字符串。
    这是一个简单的示例函数，常用于编程初学者的第一个程序示例。

    返回值:
        无返回值，直接通过print函数输出到控制台。
    """
    print("Hello world")



## 4. 借助大模型实现统计所有邮件个数的函数 

In [74]:
def get_latest_email(userId):
    """
    获取最新的一封邮件
    参数:
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{user_id}' ORDER BY date DESC LIMIT 1";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")  
    

In [75]:
code = inspect.getsource(get_latest_email)

In [76]:
code

'def get_latest_email(userId):\n    """\n    获取最新的一封邮件\n    参数:\n    user_id: 要检索邮件的用户的ID。默认值是\'me\'，数据格式为字符串类型。\n\n    返回:\n    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：\n    \'from_userid\': 发件人ID。\n    \'date\': 邮件的发送日期。\n    \'subject\': 邮件的主题。\n    \'snippet\': 邮件的摘要（前100个字符）。\n    """\n    mysql_pw="gpt"\n    \n    connection = pymysql.connect(\n    host=\'localhost\',  # 数据库地址\n    user=\'gpt\',  # 数据库用户名\n    passwd=mysql_pw,  # 数据库密码\n    db=\'my_mail\',  # 数据库名\n    charset=\'utf8\'  # 字符集选择utf8   \n    )\n    sql_query=f"SELECT * FROM mailbox WHERE userid = \'{user_id}\' ORDER BY date DESC LIMIT 1";\n\n    try:\n        with connection.cursor() as cursor:\n            sql = sql_query\n            cursor.execute(sql)\n            results = cursor.fetchall()\n    finally:\n        cursor.close()\n    \n    column_names = [desc[0] for desc in cursor.description]\n\n    # 使用results和column_names创建DataFrame\n    df = pd.DataFrame(results, columns=column_names)\n        \n    return df.t

In [77]:
# 写入本地
with open('%s.py' % 'code/get_latest_email', 'w', encoding='utf-8') as f:
    f.write(code)

In [79]:
with open('%s.py' % 'code/get_latest_email', encoding='utf-8') as f:
    content = f.read()

In [81]:
print(content)

def get_latest_email(userId):
    """
    获取最新的一封邮件
    参数:
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{user_id}' ORDER BY date DESC LIMIT 1";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")  



接下来是具体的Few-shot提示过程：

In [99]:
# 打开并读取Markdown文件
with open('/root/autodl-tmp/面向目标架构落地/data/my_mail数据字典.md', 'r', encoding='utf-8') as f:
    md_content = f.read()
    
md_content

'# my_mail数据字典：my_mail数据库数据字典\n\n本数据字典记录了my_mail数据库中各张数据表的基本情况。\n我的my_mail中\nhost=\'localhost\',\nuser=\'gpt\',\npasswd="gpt",\ndb=\'my_mail\'\n\n## 1.mailbox数据表\n\n- 基本解释\n\n  \u200b\t\tmailbox数据表记录了邮箱里的信息，包含收件人，发件人，发件时间，邮件主题，邮件内容，邮件状态，邮件标签等。\n\n- 数据来源\n\n  \u200b\tmailbox数据集由系统记录，并且校验过相关信息，数据集的准确性和可信度都非常高。\n\n- 各字段说明\n\n| Column Name | Description | Value Range | Value Explanation | Type |\n|-------------|-------------|-------------|-------------------|------|\n| userid | 收件人 |              | 由字符串组成的 | VARCHAR(255) |\n| from_userid | 发件人 |  | 由字符串组成 | VARCHAR(255) |\n| date | 发邮件的时间 |  | 日期格式  | VARCHAR(255) |\n| subject | 邮件主题 |  | 由字符串组成 | VARCHAR(255) |\n| snippet | 邮件内容 | | 由字符串组成 | VARCHAR(255) |\n| status | 邮件状态 | 已读，未读 | 已读，未读 | VARCHAR(255) |\n| label | 邮件标签 |  | 由字符串组成| VARCHAR(255) |'

In [92]:
user_example_content = "请帮我编写一个python函数，用于查看我的mailbox邮箱中最后一封邮件信息，函数要求如下：\
                        1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                        2.函数返回结果是一个包含最后一封邮件信息的对象，返回结果本身必须是一个json格式对象；\
                        3.请将全部功能封装在一个函数内；\
                        4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [93]:
with open('%s.py' % 'code/get_latest_email', encoding='utf-8') as f:
    assistant_example_content = f.read()

In [94]:
print(assistant_example_content)

def get_latest_email(userId):
    """
    获取最新的一封邮件
    参数:
    user_id: 要检索邮件的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个列表，其中每个元素都是一个字典，表示一封邮件。每个字典包含以下键：
    'from_userid': 发件人ID。
    'date': 邮件的发送日期。
    'subject': 邮件的主题。
    'snippet': 邮件的摘要（前100个字符）。
    """
    mysql_pw="gpt"
    
    connection = pymysql.connect(
    host='localhost',  # 数据库地址
    user='gpt',  # 数据库用户名
    passwd=mysql_pw,  # 数据库密码
    db='my_mail',  # 数据库名
    charset='utf8'  # 字符集选择utf8   
    )
    sql_query=f"SELECT * FROM mailbox WHERE userid = '{user_id}' ORDER BY date DESC LIMIT 1";

    try:
        with connection.cursor() as cursor:
            sql = sql_query
            cursor.execute(sql)
            results = cursor.fetchall()
    finally:
        cursor.close()
    
    column_names = [desc[0] for desc in cursor.description]

    # 使用results和column_names创建DataFrame
    df = pd.DataFrame(results, columns=column_names)
        
    return df.to_json(orient = "records")  



In [95]:
user_content = "请帮我编写一个python函数，用于查看我的mailbox邮箱中总共有多少封邮件，函数要求如下：\
                1.函数参数userId，userId是字符串参数，默认情况下取值为'me'，表示查看我的邮件；\
                2.函数返回结果是当前邮件总数，返回结果本身必须是一个json格式对象；\
                3.请将全部功能封装在一个函数内；\
                4.请在函数编写过程中，在函数内部加入中文编写的详细的函数说明文档，用于说明函数功能、函数参数情况以及函数返回结果等信息；"

In [114]:
## 2个以上 one-shot few-shot 2个或者以上
messages=[{"role": "system", "content": md_content},
          {"role": "user", "name":"example_user", "content": user_example_content},
          {"role": "assistant", "name":"example_assistant", "content": assistant_example_content},
          {"role": "user", "name":"example_user", "content": user_content}]

In [115]:
response = client.chat.completions.create(
                        model="glm-4",
                        messages=messages)

content = response.choices[0].message.content

In [116]:
print(content)

下面是一个Python函数，用于查询指定用户邮箱中的邮件总数，并将结果以JSON格式返回：

```python
import pymysql
import json

def get_email_count(userId='me'):
    """
    获取指定用户邮箱中的邮件总数。

    参数:
    userId: 要查询的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个JSON格式对象，包含一个键值对，键为'count'，值为邮件总数。

    示例:
    {
        "count": 123
    }
    """

    # 数据库连接配置
    config = {
        'host': 'localhost',
        'user': 'gpt',
        'passwd': 'gpt',
        'db': 'my_mail',
        'charset': 'utf8'
    }

    # 建立数据库连接
    try:
        connection = pymysql.connect(**config)
        with connection.cursor() as cursor:
            # 查询邮件总数
            sql = f"SELECT COUNT(*) FROM mailbox WHERE userid = '{userId}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            email_count = result[0] if result else 0

            # 将结果转换为JSON格式
            return json.dumps({'count': email_count})
    except pymysql.MySQLError as e:
        print(f"数据库操作出错: {e}")
        return json.dumps({'error': str(e)})
    fin

In [117]:
extract_function_code(content, detail=0)

The function name is:get_email_count


In [118]:
extract_function_code(content, detail=1)

def get_email_count(userId='me'):
    """
    获取指定用户邮箱中的邮件总数。

    参数:
    userId: 要查询的用户的ID。默认值是'me'，数据格式为字符串类型。

    返回:
    一个JSON格式对象，包含一个键值对，键为'count'，值为邮件总数。

    示例:
    {
        "count": 123
    }
    """

    # 数据库连接配置
    config = {
        'host': 'localhost',
        'user': 'gpt',
        'passwd': 'gpt',
        'db': 'my_mail',
        'charset': 'utf8'
    }

    # 建立数据库连接
    try:
        connection = pymysql.connect(**config)
        with connection.cursor() as cursor:
            # 查询邮件总数
            sql = f"SELECT COUNT(*) FROM mailbox WHERE userid = '{userId}'"
            cursor.execute(sql)
            result = cursor.fetchone()
            email_count = result[0] if result else 0

            # 将结果转换为JSON格式
            return json.dumps({'count': email_count})
    except pymysql.MySQLError as e:
        print(f"数据库操作出错: {e}")
        return json.dumps({'error': str(e)})
    finally:
        if connection:
            connection.close()

# 示例调用
# print(get_email

In [119]:
get_email_count?

Signature: get_email_count(userId='me')
Docstring:
获取指定用户邮箱中的邮件总数。

参数:
userId: 要查询的用户的ID。默认值是'me'，数据格式为字符串类型。

返回:
一个JSON格式对象，包含一个键值对，键为'count'，值为邮件总数。

示例:
{
    "count": 123
}
File:      Dynamically generated function. No source code available.
Type:      function

In [120]:
get_email_count(userId='me')

'{"count": 3}'

上面的答案计算正确，我们把这个函数看做就是一个我们项目里的一个方法，专门用来统计某个人一共有多少封邮件的功能。

接下来我们看一下大模型自动生成的这个外部函数，能不能被正常调用。

In [121]:
functions_list = [get_email_count]

In [122]:
functions = auto_functions(functions_list)
functions

[{'type': 'function',
  'function': {'name': 'get_email_count',
   'description': '获取指定用户邮箱中的邮件总数。',
   'parameters': {'type': 'object',
    'properties': {'userId': {'type': 'string',
      'description': "要查询的用户的ID。默认值是'me'。",
      'default': 'me'}},
    'required': ['userId'],
    'additionalProperties': False}}}]

In [124]:
def chat_with_model(functions_list=None, 
                    prompt="你好", 
                    model="glm-4", 
                    system_message=[{"role": "system", "content": "你是小智助手。"}]):
    
    messages = system_message
    messages.append({"role": "user", "content": prompt})
    
    while True:           
        answer = run_conversation(messages=messages, 
                                    functions_list=functions_list, 
                                    model=model)
        
        print(f"智能助手回答: {answer}")
        
        
        # 询问用户是否还有其他问题
        user_input = input("您还有其他问题吗？(输入退出以结束对话): ")
        if user_input == "退出":
            break

        # 记录用户回答
        messages.append({"role": "user", "content": user_input})

In [125]:
chat_with_model(functions_list=functions_list)

{'get_email_count': <function get_email_count at 0x7ff85236b550>}
智能助手回答: ，我是人工智能助手，可以叫我小智🤖，很高兴见到你，欢迎问我任何问题。


您还有其他问题吗？(输入退出以结束对话):  请帮我查询一下，邮箱里我的邮件有多少封？


{'get_email_count': <function get_email_count at 0x7ff85236b550>}
智能助手回答: 好的，我可以帮你查询邮件总数。请问你的用户ID是多少呢？


您还有其他问题吗？(输入退出以结束对话):  me


{'get_email_count': <function get_email_count at 0x7ff85236b550>}
CompletionMessage(content='您邮箱里的邮件总数是3封。', role='assistant', tool_calls=None)
智能助手回答: 您邮箱里的邮件总数是3封。


您还有其他问题吗？(输入退出以结束对话):  退出


至此，我们就完成了统计邮件总数的函数编写。当前外部函数编写的效率在extract_function_code函数加持下已经非常高了，我们只需要少量的人工来编写prompt、并围绕新函数进行测试，即可完成外部函数的编写。

&emsp;&emsp;但是，大家有没有想过，当前AI应用的开发流程其实还可以更加高效一些——面对大量潜在的未知用户需求（比如现在我想查看下邮箱里是否有陆小凤发来的未读邮件），我们或许可以借助大语言模型，即时将用户的需求翻译成外部函数创建的prompt，然后即时创建外部函数加入到Chat模型中来实时更新Chat模型能力，并最终提供更加完善的解决方案。这其实是一种更高级的自动化形式——大模型不仅可以编写功能实现的代码，而且需求到功能的过程也可以由大模型自己来进行总结。而若能做到这一点，此时这个产品的功能就相当于是可以实现自生长（根据用户的需求实时成长），毫无疑问，这样的一个开发过程，才更加贴近我们想象中的智能化开发过程。

智能化的路上我们一起加油！